In [40]:
import cv2
import numpy as np

In [41]:
def to_grayscale(img):
    height, width, _ = img.shape
    grayscale_img = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            pixel = img[i, j]
            gray_value = 0.11 * pixel[0] + 0.53 * pixel[1] + 0.36 * pixel[2]
            grayscale_img[i, j] = gray_value

    return grayscale_img


image = to_grayscale(cv2.imread('input_photos/img.png'))
image = (image > 127).astype(np.uint8)

In [42]:
kernel = np.array([[0, 1, 0],
                   [1, 1, 1],
                   [0, 1, 0]], np.uint8)

In [43]:
def prepare_for_convolution(img, krnl):
    img_h, img_w = img.shape
    kernel_h, kernel_w = krnl.shape
    padding_h, padding_w = kernel_h // 2, kernel_w // 2

    padded_img = np.pad(img, ((padding_h, padding_h), (padding_w, padding_w)), mode='constant')
    output_img = np.zeros_like(img)

    return img_h, img_w, padded_img, output_img, kernel_h, kernel_w

In [44]:
def erosion(img, krnl):

    h, w, padded_img, result, k_h, k_w = prepare_for_convolution(img, krnl)

    for i in range(h):
        for j in range(w):
            l = padded_img[i:i + k_h, j:j + k_w]

            if np.all(l[krnl == 1] == 1):
                result[i, j] = 1

    return result

cv2.imwrite('output_photos/erosion_image.png', erosion(image, kernel)*255)

True

In [45]:
def dilation(img, k):
    h, w, padded_img, result, k_h, k_w = prepare_for_convolution(img, k)

    for i in range(h):
        for j in range(w):

            l = padded_img[i:i + k_h, j:j + k_w]

            if np.any(l[kernel == 1] > 0):
                result[i, j] = 1

    return result

cv2.imwrite('output_photos/dilation_image.png', dilation(image, kernel)*255)

True

In [46]:
def opening(img, krnl):
    return dilation(erosion(img, krnl), krnl)

cv2.imwrite('output_photos/opening_image.png', opening(image, kernel)*255)

True

In [47]:
def closing(img, krnl):
    return erosion(dilation(img, krnl), krnl)

cv2.imwrite('output_photos/closing_image.png', closing(image, kernel)*255)

True

In [48]:
def boundaries(img, krnl):
    erosion_img = erosion(img, krnl)
    result = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if image[i, j] == 1 and erosion_img[i, j] == 0:
                result[i, j] = 1

    return result

cv2.imwrite('output_photos/boundaries_image.png', boundaries(image, kernel)*255)

True